# From raw catalog in JSON to catalog in JSONL suitable for BigQuery

## Preliminary steps

### Install the libraries used in this tutorial

Library to authenticate with Google Cloud products


In [150]:
pip install --upgrade google-auth

Note: you may need to restart the kernel to use updated packages.


### Import the necessary libraries

Google Cloud client library for Cloud Storage (GCS)

In [151]:
from google.cloud import storage

### Define global project variables and credentials

In [166]:
# The name for the new bucket. Change the bucker name to an unique bucket name.
bucket_name = "pod-fr-retail-retailsearch-asset"
# The name for the file to import from GCS
source_blob_name = "products.json"
# The name for the JSONL file to write back in GCS
destination_blob_name = "products_bq.json"

### Define functions

Function to read content of a blob in GCS and parse as a JSON (https://cloud.google.com/storage/docs/downloading-objects#client-libraries-download-object)

In [165]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your GCS object
    # source_blob_name = "storage-object-name"

    # The path to which the file should be downloaded
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Downloaded storage object {} from bucket {} to local file {}.".format(
            source_blob_name, bucket_name, destination_file_name
        )
    )
    

Function to write content into a blob and upload it to GCS (https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python)

In [154]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

## Process

Download blob from GCS

In [178]:
products_blob = download_blob(bucket_name, source_blob_name, source_blob_name)

Downloaded storage object products.json from bucket pod-fr-retail-retailsearch-asset to local file products.json.


Open file and parse as JSON

In [179]:
products = json.load(open(source_blob_name, 'r'))

Transform categories into Retail API format (see https://cloud.google.com/retail/docs/reference/rest/v2/projects.locations.catalogs.branches.products)

In [180]:
for product in products:
    product["category"] = (' > '.join(category["name"] for category in product["category"]))

Transform JSON into JSONL format (see https://stackoverflow.com/questions/51300674/converting-json-into-newline-delimited-json-in-python/51301075#51301075)

In [181]:
contents = ('\n'.join([json.dumps(record) for record in products]))

Write contents into a file

In [182]:
output = open(destination_blob_name, 'w')
output.write(contents)

35089846

Upload file to GCS

In [183]:
upload_blob(bucket_name, destination_blob_name, destination_blob_name)

File products_bq.json uploaded to products_bq.json.
